<!--
SPDX-FileCopyrightText: Copyright (c) 2024 Idiap Research Institute <contact@idiap.ch>
SPDX-FileContributor: Alina Elena Baia <alina.baia.idiap.ch>
SPDX-FileContributor: Darya Baranouskaya <darya.baranouskaya.idiap.ch>
SPDX-FileContributor: Olena Hrynenko <olena.hrynenko.idiap.ch>
-->

# Practice exercises after Lecture 5
This notebook contains the practice exercise with instructions and explanations.

Work through the cells below in sequential order, executing each cell as you progress. Throughout the notebook, you will encounter instructions marked with the words **YOUR CODE HERE** followed by **raise NotImplementedError()**. You will have to substitute  *raise NotImplementedError()* with your own code.
Follow the instructions and write the code to complete the tasks.

Along the way, you will also find questions. Try to reflect on the questions before/after running the code.


You will get familiar with HuggingFace, a "platform where the machine learning community collaborates on models, datasets, and applications".  [[cit. HuggingFace main page]](https://huggingface.co/).

You will learn how to finetune a transformer-based model for a binary text classification task using a pretrained model avaiable on HuggingFace. Specifically, you will finetune a transformer to classify movie reviews, from [IMDB dataset](https://huggingface.co/datasets/imdb), as either positive or negative reviews.

Moreover, the goal of this exercise is to learn  how to use HuggingFace for fast model prototyping which allows you to quickly experiment with different architectures, adapt exsiting pretrained models and create baseline models for custom tasks  (for example, the Mini-Project problem).


This notebook was developed at the [Idiap Research Institute](https://www.idiap.ch) by [Alina Elena Baia](mailto:alina.baia.idiap.ch>), [Darya Baranouskaya](mailto:darya.baranouskaya.idiap.ch) and [Olena Hrynenko](mailto:olena.hrynenko.idiap.ch) (equal contribution). This notebook is based on the [HuggingFace fine-tuning tutorial](https://huggingface.co/docs/transformers/training#fine-tune-a-pretrained-model).


**Why using a pretrained model?**

"There are significant benefits to using a pretrained model. It reduces computation costs, your carbon footprint, and allows you to use state-of-the-art models without having to train one from scratch. Huggingface Transformers provides access to thousands of pretrained models for a wide range of tasks. When you use a pretrained model, you train it on a dataset specific to your task. This is known as fine-tuning, an incredibly powerful training technique. " [[cit. HuggingFace fine-tuning tutorial]](https://huggingface.co/docs/transformers/training#fine-tune-a-pretrained-model).

This notebook uses [BERT-tiny model](https://github.com/google-research/bert/blob/master/README.md), named "google/bert_uncased_L-2_H-128_A-2". We use it for teaching purposes as it requires fewer computational resources and it allows fast training on CPU.

NOTE: The code can be easily adapted to use more performant models by changing the model name from the list of [available models](https://huggingface.co/docs/transformers/model_doc/auto) (i.e. DistilBERT or BERT). However GPU access and additional training time is needed when using bigger models.


Therefore, you will:

  - 5.1 Get familiar with HuggingFace datasets and tokenizers.

  - 5.2 Load and customize a HuggingFace pretrained transformer for text classification.

  - 5.3 Fine-tune and evaluate the transformer.

  - 5.4 Use the fine-tuned model for inference.

First, install all the necessary HuggingFace libraries:
  - transformers: "provides APIs and tools to easily download and train state-of-the-art pretrained models" [[cit. HuggingFace transformers]](https://huggingface.co/docs/transformers/en/index)
  - datasets: "library for easily accessing and sharing datasets for Audio, Computer Vision, and Natural Language Processing (NLP) tasks". Allows to "load a dataset in a single line of code, and use our powerful data processing methods to quickly get your dataset ready for training a deep learning model." [[cit. HuggingFace datasets]](https://huggingface.co/docs/datasets/en/index)
  - evaluate: "library for easily evaluating machine learning models and datasets. With a single line of code, you get access to dozens of evaluation methods for different domains (NLP, Computer Vision, Reinforcement Learning, and more!)." [[cit. HuggingFace evaluate]](https://huggingface.co/docs/evaluate/en/index)

NOTE: if you want to "enable the same PyTorch code to be run across any distributed configuration" you can use [HuggingFace accelerate](https://huggingface.co/docs/accelerate/en/index) library. You will **NOT** be using accelerate in this notebook.

  <!-- -accelerate: to install HuggingFace accelerate "library that enables the same PyTorch code to be run across any distributed configuration by adding just four lines of code! In short, training and inference at scale made simple, efficient and adaptable." [cit. https://huggingface.co/docs/accelerate/en/index] -->

In [1]:
import getpass
import os
import re
import torch

# For efficient usage of the hardware resources when running on JupyterHub EPFL,
# we will limit the number of threads. If you are running this code on your local
# machine or on colab, the following code will not do anything.

if getpass.getuser() == "jovyan":
    num_threads_limit = 4
elif re.search('^https://.*noto.*\.epfl\.ch$', os.environ.get("EXTERNAL_URL", "")) != None:
    num_threads_limit = 2
else:
    num_threads_limit = torch.get_num_threads()
print(f"Limiting the number of threads to {num_threads_limit}")
torch.set_num_threads(num_threads_limit)
print(f"PyTorch is using {torch.get_num_threads()} threads")

_ = torch.set_flush_denormal(True) # To avoid long training time on CPU

Limiting the number of threads to 1
PyTorch is using 1 threads


In [2]:
# If you are using Noto, you might need to restart the kernel after the installation
!pip install transformers datasets evaluate

# Dataset and tokenization

  - 5.1 Get familiar with HuggingFace datasets and tokenizers.


Next, prepare the IMDB dataset. You can use the HuggingFace dataset library to:
 - load a dataset from the HuggingFace Hub: "Over 1,000 datasets for many NLP tasks like text classification, question answering, language modeling, etc, are provided on the HuggingFace Hub and can be viewed and explored online with the 🤗 Datasets viewer." [[cit. HuggingFace loading datasets]](https://huggingface.co/docs/datasets/v1.11.0/loading_datasets.html). Check [this link](https://huggingface.co/datasets/viewer/) if you want to explore a dataset using a user-friendly interface.
 - load a dataset from local files: "Generic loading scripts are provided for: CSV files (with the csv script), JSON files (with the json script), text files (read as a line-by-line dataset with the text script), pandas pickled dataframe (with the pandas script)" [[cit. HuggingFace loading datasets]](https://huggingface.co/docs/datasets/v1.11.0/loading_datasets.html). For example and future reference, you can load a dataset from a local .csv file into a dataset object with the following code:
 ```
 imdb = load_dataset('csv', data_files=["<your-path-to-dataset>/IMDB-Dataset.csv"])
 ```


 In this notebook, you will load the IMDB dataset from the HuggingFace Hub and you will also learn how to adapt the dataset to be used with native Pytorch.

 IMDB "is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well." [[cit. HuggingFace IMDB dataset]](https://huggingface.co/datasets/imdb)


In [3]:
from datasets import load_dataset

#load a dataset form the HuggingFace Hub
imdb = load_dataset("imdb")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.10/dist-packages/numpy/core/getli

Let us explore the dataset and its content.

"What is in the Dataset object?

The datasets.Dataset object that you get when you execute for instance the following commands:

```
from datasets import load_dataset
dataset = load_dataset("imdb")
```
behaves like a normal python container. You can query its length, get rows, columns and also a lot of metadata on the dataset (description, citation, split sizes, etc)." [[cit. HuggingFace What’s in the Dataset object]](https://huggingface.co/docs/datasets/v1.11.0/exploring.html)


In [4]:
# check the dataset structure
print(imdb)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


NOTE: the dataset does not come with a validation split. As the goal of this exercises is to learn the process of finetuning HuggingFace models and not to find the best model that solves the task, we will not be using a validation set. However, keep in mind that for real-world problem using a validation set is important because it gives "an unbiased estimate of the skill of the final tuned model when comparing or selecting between final models" [[cit. Machine Learning Mastery Difference Between Test and Validation]](https://machinelearningmastery.com/difference-test-validation-datasets/).



In [5]:
# check the size of the dataset
# like this
print(imdb.shape)

#or like this
print(len(imdb["train"]["text"]), len(imdb["train"]["label"]))
print(len(imdb["test"]["text"]), len(imdb["test"]["label"]))

{'train': (25000, 2), 'test': (25000, 2), 'unsupervised': (50000, 2)}
25000 25000
25000 25000


In [6]:
# take a look at an example
imdb["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

There are two fields in this dataset: [[cit. HuggingFace Text classification]](https://huggingface.co/docs/transformers/tasks/sequence_classification).
 - text: the movie review text.
 - label: a value that is either 0 for a negative review or 1 for a positive review.


In [7]:
# take a look at an example
print("review: ", imdb["train"][0]["text"])
print("label: ", imdb["train"][0]["label"])

print("review: ", imdb["train"][-1]["text"])
print("label: ", imdb["train"][-1]["label"])

print(len(imdb["train"]["label"]), imdb["train"]["label"])

review:  I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far bet

In [8]:
#check class distribution in training and testing set

from collections import Counter
print(Counter(imdb["train"]["label"]))

print(Counter(imdb["test"]["label"]))

Counter({0: 12500, 1: 12500})
Counter({0: 12500, 1: 12500})


Next, load the BERT-tiny tokenizer to preprocess the text data and create a preprocessing function to tokenize text and truncate sequences to be no longer than model's maximum input length [[cit. HuggingFace Text classification]](https://huggingface.co/docs/transformers/tasks/sequence_classification).



"In many cases, the architecture you want to use can be guessed from the name or the path of the pretrained model you are supplying to the from_pretrained()
method. AutoClasses are here to do this job for you so that you automatically
retrieve the relevant model given the name/path to the pretrained weights/config/vocabulary." [[cit. HuggingFace Auto Classes]](https://huggingface.co/docs/transformers/model_doc/auto).


Details about tokenizers class can also be found at [HuggingFace tokenizer](https://huggingface.co/transformers/v2.11.0/main_classes/tokenizer.html) page:

- model_max_length (int): "the maximum length in number of tokens
  for the inputs to the transformer model. When the tokenizer is loaded with
  from_pretrained, this will be set to the value stored for the associated
  model in max_model_input_sizes (see above).
  If no value is provided, will default to VERY_LARGE_INTEGER (int(1e30)).
  no associated max_length can be found in max_model_input_sizes." [[cit. HuggingFace tokenizer]](https://huggingface.co/transformers/v2.11.0/main_classes/tokenizer.html)





In [9]:
from transformers import AutoTokenizer

#  Instantiating  AutoTokenizer will directly create a class of the relevant architecture.
tokenizer  = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2", model_max_length=512)


In [10]:
#explore the tokenizer

tokenizer

BertTokenizerFast(name_or_path='google/bert_uncased_L-2_H-128_A-2', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


Why using padding and truncation?
"Batched inputs are often different lengths, so they can’t be converted to fixed-size tensors. Padding and truncation are strategies for dealing with this problem, to create rectangular tensors from batches of varying lengths. Padding adds a special padding token to ensure shorter sequences will have the same length as either the longest sequence in a batch or the maximum length accepted by the model. Truncation works in the other direction by truncating long sequences." [[cit. HuggingFace Padding and truncation]](https://huggingface.co/docs/transformers/en/pad_truncation#padding-and-truncation)

More information about padding and truncation is available at [HuggingFace Padding and truncation](https://huggingface.co/docs/transformers/en/pad_truncation#padding-and-truncation) page.


In [11]:
def tokenize_function(examples):
    #  padding: 'max_length': pad to a length specified by the max_length argument or the
    #  maximum length accepted by the model if no max_length is provided (max_length=None).
    #  Padding will still be applied if you only provide a single sequence. [from documentation]


    #  truncation: True or 'longest_first': truncate to a maximum length specified
    #  by the max_length argument or the maximum length accepted by the model if
    #  no max_length is provided (max_length=None). This will truncate token by
    #  token, removing a token from the longest sequence in the pair until the
    #  proper length is reached. [from documentation]
    return tokenizer(examples["text"], padding="max_length", truncation=True)


#  apply the function to all the elements in the dataset (individually or in batches)
#  https://huggingface.co/docs/datasets/v1.11.0/package_reference/main_classes.html?highlight=dataset%20map#datasets.Dataset.map
#  batch mode is very powerful. It allows you to speed up processing
#  more info here: https://huggingface.co/docs/datasets/en/about_map_batch
tokenized_imdb = imdb.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Explore the tokenized dataset and check the differences in structure with respect to the original dataset.

In [12]:
# explore the tokenized dataset

# print the tokenized dataset
print(tokenized_imdb)


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})


Explore the tokenized dataset.

Can you spot the [CLS] and [SEP] tokens in the tokenized reviews?

In [13]:
# print the first review of the tokenized dataset
# 1 line of code
### BEGIN SOLUTION
print(tokenized_imdb["train"][0]["text"])
### END SOLUTION


# print the label of the first review of the tokenized dataset
# 1 line of code
### BEGIN SOLUTION
print(tokenized_imdb["train"][0]["label"])
### END SOLUTION

# print the tokens of the first review of the tokenized dataset
# Can you spot the [CLS] and [SEP] tokens in the tokenized review?
# 1 line of code
### BEGIN SOLUTION
print(tokenized_imdb["train"][0]["input_ids"])
### END SOLUTION


# print the attention_mask of the first review of the tokenized dataset

# 1 line of code
### BEGIN SOLUTION
print(tokenized_imdb["train"][0]["attention_mask"])
### END SOLUTION



I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

Look at the token_type_ids of a sample.
What do you notice?

Check [HuggingFace Glossary](https://huggingface.co/docs/transformers/en/glossary) for the explanation.

In [14]:
# print the token_type_ids of the first review of the tokenized dataset
# 1 line of code
### BEGIN SOLUTION
print(tokenized_imdb["train"][0]["token_type_ids"])
### END SOLUTION

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

So far, you used the HuggingFace dataset object to explore the dataset. We need to do some dataset manipulation in order to make the dataset compatible with native pytorch.

Therefore, manually postprocess tokenized_dataset to prepare it for training with Pytorch, following these steps:
 -  "Remove the text column because the model does not accept raw text as an input"
 - "Rename the label column to labels because the model expects the argument to be named labels"
 - "Set the format of the dataset to return PyTorch tensors instead of lists"

  [[cit. HuggingFace Train in Native PyTorch]](https://huggingface.co/docs/transformers/training#train-in-native-pytorch)

In [15]:
# before any manual postprocessing
print(tokenized_imdb)
print(type(tokenized_imdb), type(tokenized_imdb["train"]["label"]))

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})
<class 'datasets.dataset_dict.DatasetDict'> <class 'list'>


In [16]:
# Remove the text column because the model does not accept raw text as an input

tokenized_imdb = tokenized_imdb.remove_columns(["text"])
print(tokenized_imdb)
print(type(tokenized_imdb), type(tokenized_imdb["train"]["label"]))

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})
<class 'datasets.dataset_dict.DatasetDict'> <class 'list'>


In [17]:
#Rename the label column to labels because the model expects the argument to be named labels

tokenized_imdb = tokenized_imdb.rename_column("label", "labels")
print(tokenized_imdb)
print(type(tokenized_imdb), type(tokenized_imdb["train"]["labels"]))

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})
<class 'datasets.dataset_dict.DatasetDict'> <class 'list'>


In [18]:
# Set the format of the dataset to return PyTorch tensors instead of lists

tokenized_imdb.set_format("torch")
print(tokenized_imdb)
print(type(tokenized_imdb), type(tokenized_imdb["train"]["labels"]))

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})
<class 'datasets.dataset_dict.DatasetDict'> <class 'torch.Tensor'>


The dataset has 25k samples for training and 25k for testing. Using the entire training set will take a long time to finish the fine-tuning step. Therefore, for teaching purposes, we will "create a smaller subset of the original dataset to speed up the fine-tuning" [[cit. HuggingFace Train in Native PyTorch]](https://huggingface.co/docs/transformers/training#train-in-native-pytorch)

In [19]:
# create a smaller subset of the dataset as previously shown to speed up the fine-tuning

small_train_dataset = tokenized_imdb["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_imdb["test"].shuffle(seed=42).select(range(1000))

Then, "create a DataLoader for your training and test datasets so you can iterate over batches of data." [[cit. HuggingFace Train in Native PyTorch]](https://huggingface.co/docs/transformers/training#train-in-native-pytorch)

In [20]:
# create a DataLoader for your training and test datasets so you can iterate over batches of data:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [21]:
# get some random training samples
dataiter = iter(train_dataloader)
batch_data= next(dataiter)

print(batch_data)

#NOTICE CLS token =101 and SEP token = 102

{'labels': tensor([0, 1, 0, 1, 0, 1, 1, 1]), 'input_ids': tensor([[  101,  2057,  2031,  ...,     0,     0,     0],
        [  101,  2137, 11170,  ..., 23974,  1012,   102],
        [  101,  4873,  1010,  ...,     0,     0,     0],
        ...,
        [  101,  2025,  1996,  ...,     0,     0,     0],
        [  101,  1045,  2387,  ...,     0,     0,     0],
        [  101,  2013,  1996,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


# Model and optimizer

  - 5.2 Load and customize a HuggingFace pretrained transformer for text classification.



"Before you start training your model, create a map of the expected ids to their labels with id2label and label2id." [[cit. HuggingFace Text classification]](https://huggingface.co/docs/transformers/tasks/sequence_classification).

Read [HuggingFace Discussion: "Change label names on inference API"](https://discuss.huggingface.co/t/change-label-names-on-inference-api/3063) for more information on changing the label names.

In [22]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

Load the BERT-tiny with the number of expected labels using Auto classes.

Documentation of BERT-tiny available [here](https://github.com/google-research/bert/blob/master/README.md).


In [23]:
from transformers import AutoModelForSequenceClassification
# BERT-tiny license

# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


model_bert_l4 = AutoModelForSequenceClassification.from_pretrained(
    "google/bert_uncased_L-4_H-128_A-2", num_labels=2, id2label=id2label, label2id=label2id)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-4_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


What to do about this warning message: *Some weights ... were not initialized from the model checkpoint ... newly initialized* ?

"The warning is telling you that some weights were randomly initialized (here you classification head), which is normal since you are instantiating a pretrained model for a different task. It’s there to remind you to finetune your model (it’s not usable for inference directly)" [[cit. HuggingFace Discussion: "Some weights of the model were not used warning"]](https://discuss.huggingface.co/t/is-some-weights-of-the-model-were-not-used-warning-normal-when-pre-trained-bert-only-by-mlm/5672)

More discussions on this warning here:

[GitHub Discussion: "What to do about this warning message: "Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification"](https://github.com/huggingface/transformers/issues/5421)

[Stack Overflow Discussion: "Python: BERT Error - Some weights of the model checkpoint at were not used when initializing BertModel"](https://stackoverflow.com/questions/67546911/python-bert-error-some-weights-of-the-model-checkpoint-at-were-not-used-when)

NOTE: If you have GPU access and time to wait for the training to finish, you can use  [DistilBERT](https://arxiv.org/pdf/1910.01108v4.pdf), a variant of BERT.

According to the paper: *While most prior work investigated the use of
distillation for building task-specific models, we leverage knowledge distillation during the pretraining phase and show that it is possible to reduce the size of a BERT model by 40%, while retaining 97% of its language understanding capabilities and being 60% faster.*

```
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)
```
Finetuning DistilBERT on the 25k training samples of IMDB for 2 epochs will yield an accuracy of ~92%, but it will take circa 40 minutes of GPU training (when using a T4 GPU)

In [24]:
print("bert-tiny variant number of parameters: ", model_bert_l4.num_parameters())


bert-tiny variant number of parameters:  4782722


You need to define the optimizer and learning rate scheduler to fine-tune the model. Let’s use the AdamW optimizer from PyTorch:

AdamW optimizer "implements gradient bias correction as well as weight decay. The optimizer allows us to apply different hyperpameters for specific parameter groups. For example, we can apply weight decay to all parameters other than bias and layer normalization terms." [[cit. HuggingFace transformers training]](https://huggingface.co/transformers/v3.5.1/training.html)

Why use AdamW? From the [paper](https://arxiv.org/abs/1711.05101): "We empirically showed that our version of Adam with decoupled weight decay yields substantially better generalization performance than the common implementation of Adam with L2 regularization." "Our proposed decoupled weight decay has
already been adopted by many researchers, and the community has implemented
it in TensorFlow and PyTorch."


Also this paper shows that "fine-tuning with AdamW
performs substantially better than SGD on modern Vision Transformer and Con-
vNeXt models. "


In [25]:
from torch.optim import AdamW

optimizer = AdamW(model_bert_l4.parameters(), lr=5e-5)

Define a learning rate scheduler.

"One commonly used technique for training a Transformer is learning rate warm-up. This means that we gradually increase the learning rate from 0 on to our originally specified learning rate in the first few iterations. Thus, we slowly start learning instead of taking very large steps from the beginning. In fact, training a deep Transformer without learning rate warm-up can make the model diverge and achieve a much worse performance on training and testing." [[cit. UvA DL - Transformers and Multi-Head Attention ]](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial6/Transformers_and_MHAttention.html)

[Stack Overflow Discussion: "What does "learning rate warm-up" mean?"](https://stackoverflow.com/questions/55933867/what-does-learning-rate-warm-up-mean).


What is learning rate warm-up and when do we need it?
"If your data set is highly differentiated, you can suffer from a sort of "early over-fitting". If your shuffled data happens to include a cluster of related, strongly-featured observations, your model's initial training can skew badly toward those features -- or worse, toward incidental features that aren't truly related to the topic at all.

Warm-up is a way to reduce the primacy effect of the early training examples. Without it, you may need to run a few extra epochs to get the convergence desired, as the model un-trains those early superstitions." [[cit. Stack Overflow Discussion: Optimizer and scheduler for BERT fine-tuning]](https://stackoverflow.com/questions/60120043/optimizer-and-scheduler-for-bert-fine-tuning)


Linear scheduler: creates "a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer" [[cit. HuggingFace: optimizer_schedules]](https://huggingface.co/docs/transformers/en/main_classes/optimizer_schedules)

In [26]:
from transformers import get_scheduler

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
# feel free to experiment with different num_warmup_steps
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=1, num_training_steps=num_training_steps
)

Specify  the device to use: CPU or GPU (if you have access to one). Keep in mind that training on a CPU may take several hours instead of a couple of minutes, depeding on the model you are using.

In [27]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_bert_l4.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

# Training and evaluation

  - 5.3 Fine-tune and evaluate the transformer.





According to [HuggingFace documentation](https://huggingface.co/docs/transformers/en/training), "you don’t have to pass a loss argument to your models when you compile() them! Hugging Face models automatically choose a loss that is appropriate for their task and model architecture if this argument is left blank. You can always override this by specifying a loss yourself if you want to. "

In [28]:
# use the tqdm library to add a progress bar over the number of training steps
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# put the model in train mode
model_bert_l4.train()

# iterate over epochs
for epoch in range(num_epochs):
    # iterate over batches in training set
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        # **kwargs is a common idiom to allow an arbitrary number of arguments to functions
        # The **kwargs will give you all keyword arguments as a dictionary
        # https://stackoverflow.com/questions/36901/what-does-double-star-asterisk-and-star-asterisk-do-for-parameters
        #

        outputs = model_bert_l4(**batch)
        #Note that Transformers models all have a default task-relevant loss function,
        #so you don’t need to specify one unless you want to

        # get the loss form the outputs
        # in this example, the outputs are instances of subclasses of ModelOutput
        # https://huggingface.co/transformers/v4.3.0/main_classes/output.html
        # Those are data structures containing all the information returned by
        # the model, but that can also be used as tuples or dictionaries.

        # the outputs object has a loss and logits attribute
        # You can access each attribute as you would usually do,
        # and if that attribute has not been returned by the model, you will get None.
        # for instance, outputs.loss is the loss computed by the model

        # 1 line of code
        ### BEGIN SOLUTION
        loss = outputs.loss
        ### END SOLUTION

        # do the backward pass
        # 1 line of code
        ### BEGIN SOLUTION
        loss.backward()
        ### END SOLUTION

        # perform one step of the optimizer
        # 1 line fo code
        ### BEGIN SOLUTION
        optimizer.step()
        ### END SOLUTION

        # peform one step of the lr_scheduler, similar with the optimizer
        # 1 line of code
        ### BEGIN SOLUTION
        lr_scheduler.step()
        ### END SOLUTION

        # zero the gradients, call zero_grad() on the optimizer
        # 1 line of code
        #### BEGIN SOLUTION
        optimizer.zero_grad()
        ### END SOLUTION

        progress_bar.update(1)

  0%|          | 0/250 [00:00<?, ?it/s]

[HuggingFace Evaluate Metric](https://huggingface.co/evaluate-metric)

[HuggingFace Metrics](https://huggingface.co/docs/datasets/en/how_to_metrics)

To evaluate the model's performance on the test set, you can use the HuggingFace evaluate library. Keep in mind that "instead of calculating and reporting the metric at the end of each epoch, this time you’ll accumulate all the batches with add_batch and calculate the metric at the very end. " [[cit. HuggingFace Train in Native PyTorch]](https://huggingface.co/docs/transformers/training#train-in-native-pytorch).

Metric Card for [accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy).

In [29]:
import evaluate

# define the metric you want to use to evaluate your model
metric = evaluate.load("accuracy")

# put the model in eval mode
model_bert_l4.eval()
# iterate over batches of evaluation dataset
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        # pass the batches to the model and get the outputs
        # (hint: look at the training loop)
        # 1 line of code:
        # outputs = ...
        ### BEGIN SOLUTION
        outputs = model_bert_l4(**batch)
        ### END SOLUTION

    # get the logits from the outputs,
    # similar as you did for the loss in the training loop
    # 1 line of code
    # logits = ...
    ### BEGIN SOLUTION
    logits = outputs.logits
    ### END SOLUTION

    # use argmax to get the predicted class
    predictions = torch.argmax(logits, dim=-1)

    # metric.add_batch() adds a batch of predictions and references
    # Metric.add_batch() by passing it your model predictions, and the references
    # the model predictions should be evaluated against
    metric.add_batch(predictions=predictions, references=batch["labels"])

# calculate a metric by  calling metric.compute()
metric.compute()



{'accuracy': 0.662}

# Inference
  - 5.4 Use the fine-tuned model for inference.

Write your own positive and negative movie reviews and use the finetuned model to classify them.

Is the model predicting as you would expect?

In [30]:
text_pos = "This movis is awesome."
text_neg = "This was a very bad movie. It was so boring that I've fell asleep while watching it."
text_pos_neg = [text_pos, text_neg]

In [31]:
# Inference on single review

# get the prediction for the positive review

# first, use the tokenier to tokenize the input text
# 1 line of code
# inputs = ...
# hint: look at the tokenizer definition provided previously
# remember to specify the padding and trunction


### BEGIN SOLUTION
inputs = tokenizer(text_pos, return_tensors="pt",  padding = True, truncation = True)
### END SOLUTION

inputs = inputs.to(device)

with torch.no_grad():
    # pass the input to the model
    # 1 line of code
    # outputs = ...
    outputs = model_bert_l4(**inputs)

    # get the logits
    # 1 line of code
    # logits = ...
    ### BEGIN SOLUTION
    logits = outputs.logits
    ### END SOLUTION

#get the predicted id class
predicted_class_id = logits.argmax().item()
print("predicted_class_id: ", predicted_class_id)

# get the predicted class name
print(model_bert_l4.config.id2label[predicted_class_id])

# Now, get the prediction for the negetaive review
# first, use the tokenier to tokenize the input text
# 1 line of code
# inputs = ...
# hint: look at the tokenizer definition provided previously
# remember to specify the padding and trunction

### BEGIN SOLUTION
inputs = tokenizer(text_neg, return_tensors="pt",  padding = True, truncation = True)
### END SOLUTION

with torch.no_grad():
    # pass the input to the model and get the logits using a single line of code
    # also, in the same line of code rememeber to put the inputs to the correct device
    # 1 line of code
    # logits = ...
    ### BEGIN SOLUTION
    logits = model_bert_l4(**inputs.to(device)).logits
    ### END SOLUTION

#get the predicted id class
predicted_class_id = logits.argmax().item()
print("predicted_class_id: ", predicted_class_id)

# get the predicted class name
print(model_bert_l4.config.id2label[predicted_class_id])

predicted_class_id:  0
NEGATIVE
predicted_class_id:  0
NEGATIVE


In [32]:
# Inference on multiple reviews
# use text_pos_neg list of reviews

# first, use the tokenier to tokenize the input text
# 1 line of code
# inputs = ...
# hint: look at the tokenizer definition provided previously
# remember to specify the padding and trunction

### BEGIN SOLUTION
inputs = tokenizer(text_pos_neg, return_tensors="pt", padding=True, truncation = True)
### END SOLUTION

with torch.no_grad():
    # pass the input to the model and get the logits using a single line of code
    # also, in the same line of code rememeber to put the inputs to the correct device
    # 1 line of code
    # logits = ...
    ### BEGIN SOLUTION
    logits = model_bert_l4(**inputs.to(device)).logits
    ### END SOLUTION

# check out the logits
print("logits: ", logits)

#get the predicted id class
predicted_class_id = logits.argmax(dim=1)
print(predicted_class_id)

# get the predicted class name
for pred in predicted_class_id:
  print(model_bert_l4.config.id2label[pred.item()])

logits:  tensor([[ 0.0431, -0.1674],
        [ 0.1907, -0.3404]], device='cuda:0')
tensor([0, 0], device='cuda:0')
NEGATIVE
NEGATIVE


Congratulations, you have reached the end of this exercise!

Remember that if you have GPU access and time to experiment with other architectures, you can try using  DistilBERT, for example, and see how the accuracy changes.